## Language Only
Glove RNN or BERT model (WIP)

* https://towardsdatascience.com/sentiment-analysis-using-lstm-and-glove-embeddings-99223a87fe8e
* https://victordibia.com/blog/text-classification-hf-tf2/

#### The following features were used

* Number of words in title
* Number of words in body
* Number of tags (_'java', 'front-end' etc._)
* User reputation at post creation 
* Account age (in days)
* Post age (in days)

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import os
import sys
from datetime import datetime
import pickle

In [ ]:
import os
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from nltk.corpus import stopwords
import nltk

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import Normalizer

In [ ]:
import torch
# from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


import torchtext
import re
from torch.utils.data import DataLoader,TensorDataset

import pyprind
%matplotlib inline  

nltk.download('punkt')
nltk.download('stopwords')

# import spacy
# tokenizer = spacy.load('en_core_web_sm')
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, GloVe

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# DATAFILE='../input/predict-closed-questions-on-stack-overflow/train.csv'
DATAFILE='../input/predict-closed-questions-on-stack-overflow/train-sample.csv'
df = pd.read_csv(DATAFILE)

open_idx = df['OpenStatus'] == 'open'
closed_idx = (1-open_idx).astype(bool)

In [ ]:
QUESTION_SAMPLE_LIMIT = 10000
TRAIN_SAMPLE_LIMIT=7500

In [ ]:
df = df.sample(frac=1)

## Preprocessing

In [ ]:
# print(df)

print("Tag remover goes first, spacy goes second")

def remove_tags(string):
    # result = re.sub('<.*?>','',string)
    try:
        result = re.sub(r"(, '[\W\.]')",r"",string)
        return result
    except:
#         print(string)
        return "invalid sentence"

text_df = pd.concat( [df['Title'].apply(lambda x: remove_tags(x) ), df['BodyMarkdown'].apply(lambda x: remove_tags(x) )] , axis=1)

#text_df

In [ ]:
# examples = ['chip', 'baby', 'Beautiful']
# vec = GloVe(name='6B', dim=50)
# # vec = GloVe(name='840B.300d', dim=300)
# ret = vec.get_vecs_by_tokens(examples, lower_case_backup=True)

In [ ]:
#Add Title before Question Body
text_df['Text']=text_df['Title']+' '+text_df['BodyMarkdown']
text_df.drop(['Title', 'BodyMarkdown'], axis=1, inplace=True)

In [ ]:
#Tokenize First
tokenizer = get_tokenizer("spacy",'en') #'en_core_web_sm'

GLOVE_NAME = '6B'
EMBED_DIM = 50
# GLOVE_NAME = '840B'
# EMBED_DIM = 300

#Word Encoder
vec = GloVe(name=GLOVE_NAME, dim=EMBED_DIM)
print("Switch to 840B.300d when you are ready")

### Investigate Sequence Lengths

In [ ]:
# # QUESTION_SAMPLE_LIMIT=50000
# QUESTION_SAMPLE_LIMIT=None

# textblocks = text_df['Text']
# if QUESTION_SAMPLE_LIMIT!=None and QUESTION_SAMPLE_LIMIT > 0:
#     textblocks = textblocks[0:QUESTION_SAMPLE_LIMIT]

# tokens = [tokenizer(x) for x in textblocks]
# # N x ? x EMBED_DIM

In [ ]:
# token_lengths = pd.DataFrame({'token_lengths': [len(x) for x in tokens]})

In [ ]:
# token_lengths = pd.DataFrame([0,1,2])
# pd.DataFrame({'length':token_lengths.to_numpy()})
# token_lengths

In [ ]:
#token_lengths

In [ ]:
# text_df

In [ ]:
# MAX_SEQ_LENGTHS = [500,250,100]
# for MAX_SEQ_LENGTH in MAX_SEQ_LENGTHS:
#     P=100*len( token_lengths[ token_lengths['token_lengths'] <= MAX_SEQ_LENGTH ] )/QUESTION_SAMPLE_LIMIT
#     print( f"Questions under {MAX_SEQ_LENGTH} tokens: ~{P}%" )

MAX_SEQ_LENGTH = 250
print( f"SET MAX_SEQ_LENGTH to {MAX_SEQ_LENGTH}")

### Tokenise

In [ ]:
# 
# .sample(frac=1)

print("REMEMBER TO UPDATE QUESTION_SAMPLE_LIMIT")

# test = Tokenizer(X['Text'][0])
# print(test)
#tokenizer.fit_on_texts(X['Text'])
# tokenizer.fit_on_texts(X)

# QUESTION_SAMPLE_LIMIT=10000
QUESTION_SAMPLE_LIMIT=None

textblocks = text_df['Text']
if QUESTION_SAMPLE_LIMIT!=None and QUESTION_SAMPLE_LIMIT > 0:
    textblocks = textblocks[0:QUESTION_SAMPLE_LIMIT]


tokenblocks = [tokenizer(x) for x in textblocks]

# len(Encodings)
# len(Encodings[0])
# len(Encodings[0])

In [ ]:
# del text_df

### Truncate if above limit

In [ ]:
token_lengths = [len(x) for x in tokenblocks]

In [ ]:
# Target Shape: N x MAX_SEQ_LENGTHS x 50

for i in range(len(tokenblocks)):
    tokenblocks[i] = tokenblocks[i][0:min(MAX_SEQ_LENGTH, len(tokenblocks[i]))]

In [ ]:
#get encodings
wEncodings = [ vec.get_vecs_by_tokens(tk, lower_case_backup=True) for tk in tokenblocks ]

### [SKIPPED] Encode global (non-sequential) features

In [ ]:
# # 0 if open, 1 if closed
df['y'] = 1-(df.OpenStatus == 'open').astype(int)

# # Encode 'post creation date' as its age in days
# df.PostCreationDate = pd.to_datetime(df.PostCreationDate)
# origion_date = max(df.PostCreationDate)
# df['PostAge'] = (origion_date-df.PostCreationDate).dt.days

# # Encode OwnerCreationDate as account age in days
# df.OwnerCreationDate = pd.to_datetime(df.OwnerCreationDate)
# origion_date = max(df.OwnerCreationDate)
# df['AccountAge'] = (origion_date-df.OwnerCreationDate).dt.days

# # Encode title and body text as their length
# df['TitleLength'] = df.apply(lambda x: len(x['Title'].split()), 1)
# df['BodyLength'] = df.apply(lambda x: len(x['BodyMarkdown'].split()), 1)

# # Number of tags a post has (between 0-5)
# df['TotalTags'] = df.apply(lambda x: sum([not pd.isnull(x[f'Tag{a}']) for a in range(1, 6)]), 1)

# features = ['PostAge', 'AccountAge', 'ReputationAtPostCreation', 'OwnerUndeletedAnswerCountAtPostTime', 'TitleLength', 'BodyLength', 'TotalTags']


### Train/Test Split

In [ ]:
# TRAIN_SAMPLE_LIMIT=200
print("REMEMBER TO UPDATE AND/OR REMOVE TRAIN_SAMPLE_LIMIT")
if TRAIN_SAMPLE_LIMIT!=None and TRAIN_SAMPLE_LIMIT > 0:
    N = min(TRAIN_SAMPLE_LIMIT,len(wEncodings))
else:
    N = len(wEncodings)

# Padding will be handled HERE
# copy these tokens into a numpy array of originally all 0s

# N x MAX_SEQ_LENGTH x EMBED_DIM
X_lan = np.zeros((N,MAX_SEQ_LENGTH,EMBED_DIM))
X_lan.shape

for i in range(N):
    enc = wEncodings[i]
    X_lan[i][0:0+len(enc)] = enc

In [ ]:
#X_meta = df[features][0:N]
y = df['y'][0:N]
#X_meta = df[features]
# y = df['y']

# X_lan = wEncodings[0:N]

In [ ]:
X_indices = list(range(N))

In [ ]:
X_train_indices, X_test_indices, y_train, y_test = train_test_split(X_indices, y, stratify=y, test_size=0.2, random_state=1)
#Use Indices Instead, for X, thank me later

X_lan_train = X_lan[X_train_indices]
X_lan_test = X_lan[X_test_indices]


In [ ]:
#Normalize Global (non-sequential) Features
# transformer = Normalizer().fit(X_train[global])
# X_train[global] = transformer.transform(X_train[global])
# X_test[global] = transformer.transform(X_test[global])

In [ ]:
X_lan_train[:1]

In [ ]:
# X_train
#y_train_tensor.shape
# tokenblocks[X_train_indices[1]]

## Model Training

In [ ]:
# add pytorch model here

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
X_lan_train_tensor = torch.tensor(X_lan_train).type(torch.FloatTensor).to(device)
X_lan_test_tensor = torch.tensor(X_lan_test).type(torch.FloatTensor).to(device)
# y_train_tensor = torch.tensor(y_train.values).to(device)
# y_test_tensor = torch.tensor(y_test.values).to(device)
y_train_tensor = torch.tensor(y_train.values).type(torch.FloatTensor).to(device)
y_test_tensor = torch.tensor(y_test.values).type(torch.FloatTensor).to(device)

In [ ]:
# X_lan_train_tensor.shape, y_train_tensor.shape
train_dataset = TensorDataset(X_lan_train_tensor,y_train_tensor)
test_dataset = TensorDataset(X_lan_test_tensor,y_test_tensor)

In [ ]:
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
#         #Identical implementation to https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
#         #Plus ReLU
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(X.shape[1], 64),
#             nn.ReLU(),
#             nn.Linear(64, 128),
#             nn.ReLU(),
#             nn.Linear(128, 2),
#             nn.ReLU(),
#             nn.Softmax(dim=1)
#             # nn.Linear(10, 20),
#             # nn.ReLU(),
#             # nn.Linear(20, 2),
#             # nn.ReLU(),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits

In [ ]:
# hidden_size
LSTM_HIDDEN = EMBED_DIM

class QuestionClassifier(nn.Module):
    def __init__(self, num_features, hidden_layer):
        super(QuestionClassifier, self).__init__()
        #Encoder
        self.lstm = nn.LSTM(num_features, hidden_layer, batch_first = True)
        self.label = nn.Linear(hidden_layer, 1)
#         self.out, (self.h,self.c) = self.lstm(x)

    def forward(self, embedded_sentence, apply_sigmoid=False):
        #embeds = self.word_embeddings(sentence)
        # lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        # print(embedded_sentence.shape)
#         lstm_out, _ = self.lstm(embedded_sentence)
#         tag_space = self.label(lstm_out)
#         tag_scores = F.log_softmax(tag_space, dim=1)
        # tag_scores = F.softmax(tag_space,dim=1)
#         return tag_scores
        # return tag_scores
        self.out, (self.h,self.c) = self.lstm(embedded_sentence)
#         return F.log_softmax(self.h, dim=1)
        tag_space = self.label(self.h)
        tag_score = F.sigmoid(tag_space)#.type(torch.float)
        return tag_score

model = QuestionClassifier(EMBED_DIM, LSTM_HIDDEN).to(device)
print(model)

In [ ]:
# model(X_temp2)

In [ ]:
# X_temp = torch.rand(10, 7, 1, device=device)
# logits = model(X_temp)
# X_temp2 = torch.tensor(X.values).type(torch.FloatTensor)
# logits = model(X_temp2)
# X_temp3 = torch.tensor(X_train).type(torch.FloatTensor)
# logits = model(X_temp3)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")

In [ ]:
BATCH_SIZE = 16
print(f"Specifying Batch Size as {BATCH_SIZE}")

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
REPORT_EVERY_N_BATCHES = 100
eps = 1e-7

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)[0]
#         print(pred)
#         print(pred.shape, y.view(-1,1).shape)
#         print(pred.dtype, y.view(-1,1).dtype)
        
        loss = loss_fn(pred, y.view(-1,1))
        # break

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % REPORT_EVERY_N_BATCHES == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    tp, tn, fp, fn = 0,0,0,0
    precision, recall = 0,0
    f1 = 0

    with torch.no_grad():
        print_ex=True
        for X, y in dataloader:
            pred = model(X)[0]
#             print("pred")
            # print(pred.shape)
#             print(pred)
            # print(pred.mean(axis=1) > 0 )
            # break
            # print("predict?")
            # print((pred > 0))
            # break
            y_pred = (pred > 0.5).to(torch.int)
            y_true = y.view(-1,1)
#             print(y_pred.shape)
#             print(y_true.shape)
            # break
#             print('"MEAN":')
#             print(pred.mean(axis=1))
#             break
#             y_pred = (pred.mean(axis=1) >= np.log(0.5)).to(torch.int)
            if print_ex:
#                 print(pred, y_pred, y_true)
                print_ex = False

            test_loss += loss_fn(pred, y.view(-1,1)).item()


            # tp = (y_true * y_pred).sum().to(torch.float32)
            ttp = (y_true * y_pred).sum().to(torch.float32)
            tp += ttp
            ttn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
            tn += ttn
            fp += ((1 - y_true) * y_pred).sum().to(torch.float32)
            fn += (y_true * (1 - y_pred)).sum().to(torch.float32)

            correct += (ttp+ttn).type(torch.float).sum().item()
            # print("y_pred")
            # print(y_pred)
            # print("y_true")
            # print(y_true)
            

    test_loss /= num_batches
    correct /= size

    precision = tp / (tp + fp + eps)
    recall = tp / (tp + fn + eps)
    f1 = 2 * precision * recall / (precision+recall+eps)


    print(f"Test Error: \n Accuracy: {(100*correct):>0.3f}%, Avg loss: {test_loss:>8f} \n")
    # print(y_pred, y_true)
    # print(tp, tn, fp, fn, correct, size)
    # print(tp, correct, size)
    acc = 100*correct
    # print(tp, tn, acc, correct)
    return (acc, test_loss, precision, recall, f1)

In [ ]:
now = datetime.now()

# dd/mm/YY H:M:S
current_time = now.strftime("%d-%m-%H-%M")


def save_performance(epoch):

    torch.save(model, f'output/models/lan_model_{current_time}_ep_{epoch}.pth')
    torch.save(model.state_dict(), f'output/models/lan_model_state_{current_time}_ep_{epoch}.pth')

    with open(f'output/performances/acc_{current_time}_ep_{epoch}.pkl', 'wb') as filehandler:
        pickle.dump(epoch_accuracies, filehandler)
    with open(f'output/performances/loss_{current_time}_ep_{epoch}.pkl', 'wb') as filehandler:
        pickle.dump(epoch_losses, filehandler)
    with open(f'output/performances/precisions_{current_time}_ep_{epoch}.pkl', 'wb') as filehandler:
        pickle.dump(epoch_precisions, filehandler)
    with open(f'output/performances/recalls_{current_time}_ep_{epoch}.pkl', 'wb') as filehandler:
        pickle.dump(epoch_recalls, filehandler)
    with open(f'output/performances/f1s_{current_time}_ep_{epoch}.pkl', 'wb') as filehandler:
        pickle.dump(epoch_f1s, filehandler)

In [ ]:
!mkdir output
!mkdir output/models
!mkdir output/performances

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
print(class_weights)

In [ ]:
EPOCHS = 1000
SAVE_EVERY_N_EPOCHS = 50
LEARNING_RATE = 1e-6

loss_fn = nn.BCELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

epoch_accuracies = []
epoch_losses = []
epoch_precisions = []
epoch_recalls = []
epoch_f1s = []
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    epoch_acc, epoch_avg_loss, epoch_precision, epoch_recall, epoch_f1 = test_loop(test_dataloader, model, loss_fn)
    epoch_accuracies.append(epoch_acc)
    epoch_losses.append(epoch_avg_loss)
    epoch_precisions.append(epoch_precision)
    epoch_recalls.append(epoch_recall)
    epoch_f1s.append(epoch_f1)
    
    if (t+1)%SAVE_EVERY_N_EPOCHS==0:
        save_performance(t+1)

print("Done!")

## Save Model + Performance

In [ ]:
# model.eval()

In [ ]:
#P

In [ ]:
save_performance('FINAL')

In [ ]:
# next(iter(train_dataloader))[0].shape

In [ ]:
# clf.score(X_test, y_test)
# X.shape[1]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=af2d1be0-78d1-4a96-889c-2df983a097e8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>